# Explore results

In this notebook, I explore the similarity results in the search for COVID-19 candidates based on existing literature data.

In [1]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import numpy as np
import collections
import h5py
import pandas as pd
from chemicalchecker.util.plot.diagnosticsplot import set_style
set_style()

## Literature data

This is how the literature data looks like once pulled from the Google Spreadsheet and processed.

In [2]:
df = pd.read_csv("../web/data/df_cand_cc_evi0_moaall.csv", sep="\t")

/opt/miniconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (11,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
with h5py.File("../web/data/dist_cc.h5", "r") as hf:
    print(hf.keys())
    ranks_raw = hf["ranks_raw"][:]
    ranks = hf["ranks"][:]
    ranks_w = hf["ranks_w"][:]
    rows = hf["rows"][:]
    cols = hf["cols"][:]

In [ ]:
np.where(rows == "WHTVZRBIWZFKQO-UHFFFAOYSA-N")

In [ ]:
np.where(cols == "WHTVZRBIWZFKQO-UHFFFAOYSA-N")

In [ ]:
np.where(cols == "DFOCUWZXJBAUSQ-URLMMPGGSA-N")

In [ ]:
ranks_w[615526,128]

In [6]:
df[(df["support"] == 2) & (df["p 1e-4"] == 1)]

,inchikey,name,is drug,evidence,moa,support,p 1e-5,p 1e-4,p 1e-3,top1_inchikey,top2_inchikey,top3_inchikey,top1_name,top2_name,top3_name
18461,XTKQDRDDLFGMQC-MGUPHCMFSA-N,Chembl39077,0,-2,-2,2,0,1,1,UCHDWCPVSPXUMX-UHFFFAOYSA-N,NaN,NaN,Montelukast,NaN,NaN
18471,XNCUQUMSNVUULG-RPGGHDGHSA-N,CID54684154,0,-2,-2,2,0,1,1,CGZOGNUFXMNYEI-UDUHKTKDSA-N,NaN,NaN,Chebi:125390,NaN,NaN
18477,HKVZHPSRFDVDKW-SFHVURJKSA-N,Chembl45962,0,-2,-2,2,0,1,1,BDSYECBSUUOPCT-UHFFFAOYSA-N,NaN,NaN,Chembl222301,NaN,NaN
18487,POTYZNMFZDFMQC-UHFFFAOYSA-N,Chembl252545,0,-2,-2,2,0,1,1,VDHAWDNDOKGFTD-UHFFFAOYSA-N,NaN,NaN,Cinacalcet,NaN,NaN
18489,CMMMTJCZUUSEPB-UHFFFAOYSA-N,CMMMTJCZUUSEPB-UHFFFAOYSA-N,0,-2,-2,2,0,1,1,MYXACKZCYFKMTE-JETRDCLMSA-N,NaN,NaN,3Spk,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37533,BCEHBSKCWLPMDN-MGPLVRAMSA-N,Voriconazole,1,-2,-2,2,0,1,1,DOMXUEMWDBAQBQ-UHFFFAOYSA-N,NaN,NaN,Terbinafine,NaN,NaN
37537,OGCKXYFATMHUQP-UHFFFAOYSA-N,Mls000713642,0,-2,-2,2,0,1,1,KBOPZPXVLCULAV-UHFFFAOYSA-N,NaN,NaN,Mesalazine,NaN,NaN
37538,NXAITCVPAUWQJG-XDJHFCHBSA-N,An-329/40484313,0,-2,-2,2,0,1,1,DSSYKIVIOFKYAU-UHFFFAOYSA-N,NaN,NaN,Synthetic camphor,NaN,NaN
37541,BAYFCNHVRYMODQ-UHFFFAOYSA-N,Chembl104313,0,-2,-2,2,0,1,1,JBPUGFODGPKTDW-SFHVURJKSA-N,NaN,NaN,Merimepodib,NaN,NaN


In [ ]:
df_lit = pickle.load(open("../outputs/df_lit_cc.pkl", "rb"))
df_lit

As you can see, some candidates were lost because they either didn't have a correct SMILES string or we couldn't find them in the CC universe of 800k molecules. Anyway, 151 rows is a lot!

## Ranking candidates

We have run the pipeline with:

* CC: Chemical Checker
* FP: Morgan Fingeprints (classical)

We compute matrices for:

* All evidence cutoffs
* All moas

Columns are:

* inchikey: candidate inchikey
* is_drug: yes (1), no (0)
* evidence: -2 (no evidence, most of the molecules), -1, 0, 1, 2, 3 (see spreadsheet)
* moa: -2 (no evidence, most of the molecules), 0, 1, 2, 3, 4, 5 (see spreadsheet)

Number of COVID molecules in:

* top_5: top 5 similarity
* lpv_5: -log p-value 5 (1e-5)
* lpv_4: -log p-value 4 (1e-4)
* lpv_3: -log p-value 3 (1e-3)
* lpv_2: -log p-value 2 (1e-2)

Finally:
* best_inchikey: COVID-19 molecule with the highest similarity to the candidate

In [ ]:
with h5py.File("../web/data/dist_cc.h5", "r") as hf:
    r = hf["ranks_raw"][:]

In [ ]:
np.min(r)

In [ ]:
def load_df(simtype, evidence, moa, sort_by):
    if evidence is None or evidence < 0:
        evi_suf = "eviall"
    else:
        evi_suf = "evi%d" % evidence
    if moa is None:
        moa_suf = "moaall"
    else:
        moa_suf = "moa%d" % moa
    fn = "../outputs/df_cand_%s_%s_%s.pkl" % (simtype, evi_suf, moa_suf)
    df = pickle.load(open(fn, "rb"))
    return df.sort_values(sort_by, ascending=False)

Based on FP similarity, considering *all* candidates and *all* moas, and focusing on a p-value of 1e-3.

In [ ]:
load_df(simtype="fp", evidence=None, moa=None, sort_by="lpv_3")

Based on CC similarities, considering only evidence >= 1 and moa = 3 (p-value of 1e-4)

In [ ]:
load_df(simtype="cc", evidence=1, moa=3, sort_by="lpv_4")

In [ ]:
def counter(df):
    a = len(df[df["lpv_4"] > 0])
    df = df[df["is_drug"] == 1]
    b = len(df[df["lpv_4"] > 0])
    print("  Chem: %d" % a)
    print("  Drug: %d" % b)

for evid in [None, 0, 1, 2, 3]:
    for moa in [None, 0, 1, 2, 3, 4, 5]:
        print("Evidence: %s - MoA: %s" % (str(evid), str(moa)))
        df = load_df(simtype="cc", evidence=evid, moa=moa, sort_by="lpv_4")
        counter(df)
    

In [ ]:
x = np.array([[1,2,3,4],[1,2,3,4], [1,2,3,4]])

In [ ]:
x*[0,1,2,3]

## Manuscript plots

## Literature table

In [ ]:
evidence_legend = {
    -1: "Failed in clinics",
    0 : "Computational",
    1 : "Preclinical",
    2 : "Clinics",
    3 : "Clinics COVID19"
}

moa_legend = {
    -2: "NA",
    0 : "Unknown",
    1 : "Host factor",
    2 : "Virus entry",
    3 : "Protease inh.",
    4 : "RNA trans./rep.",
    5 : "Immunomodulator"
}

In [ ]:
def barplot(ax, df, column):
    counts = collections.defaultdict(int)
    for r in df[column].values:
        counts[r] += 1
    keys = sorted(counts.keys())
    vals = [counts[k] for k in keys]
    if column == "moa":
        keys = [-1, 0, 1, 2, 3, 4, 5]
    ax.scatter(vals, keys, c=keys, cmap="Spectral", edgecolor="black", s=80)
    if column == "moa":
        d = moa_legend
        yticklabels = [d[x] for x in sorted(d.keys())]
        yticks = [-1, 0, 1, 2, 3, 4, 5]
        ax.set_yticks(yticks)
        ax.set_yticklabels(yticklabels)
    else:
        d = evidence_legend
        yticks = sorted(d.keys())
        ax.set_yticks(yticks)
        ax.set_yticklabels([d[x] for x in yticks])
    if column == "moa":
        ax.set_title("MoA")
    else:
        ax.set_title(column.capitalize())
    ax.set_xlabel("Molecules")

fig, axs = plt.subplots(1,2, figsize=(8,2.2))
axs = axs.flatten()

barplot(axs[0], df_lit, "evidence")
barplot(axs[1], df_lit, "moa")
plt.tight_layout()

In [ ]:
M = np.zeros((7, 5))
for i, moa in enumerate([-2, 0, 1, 2, 3, 4, 5]):
    for j, evi in enumerate([-1, 0, 1, 2, 3]):
        n = len(df_lit[(df_lit["evidence"] == evi) & (df_lit["moa"] == moa)])
        d = len(df_lit[(df_lit["evidence"] == evi) | (df_lit["moa"] == moa)])
        M[i,j] = n/d

In [ ]:
plt.imshow(M)

## Similarity matrix

Now I explore the similarity matrix

In [ ]:
with h5py.File("../web/data/dist_trim_cc.h5", "r") as hf:
    ranks = hf["ranks"][:]
    print(hf.keys())

In [ ]:
ranks.shape

In [ ]:
idxs_i = np.argsort(-np.sum(ranks, axis=1))
idxs_j = np.argsort(-np.sum(ranks, axis=0))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,3))

ax.imshow(ranks[idxs_i][:,idxs_j], aspect="auto", cmap="GnBu")

In [ ]:
rr = ranks[idxs_i][:,idxs_j]

In [ ]:
set(rr.ravel())

In [ ]:
np.sum(rr, axis=1)